In [3]:
import re
import os
import copy
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from skimage.io import imread
from sklearn.neighbors import KNeighborsClassifier as ks

In [4]:
arr=[]
labels = []
i=0
j=0
drive.mount('/content/drive')
for root,di,files in os.walk("/content/drive/MyDrive/Colab Notebooks/Face-Recognition/dataset"): 
  for name in files:
    if name.endswith("pgm"):
      root_folder = os.path.split(root)
      res=re.findall('(\d+)',root_folder[1])
      image = imread(os.path.join(root,name))
      image=image.flatten(order='C')
      arr.append(image)
      labels.append(int(res[0]))
      i=i+1
arr=np.array(arr)   
labels=np.array(labels)    

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
print(labels.shape)       
print(arr.shape)
print(labels[300])

(400,)
(400, 10304)
37


In [6]:
# adding the data to a dataframe
df = pd.DataFrame(arr) 
df['label'] = pd.DataFrame(labels)
# sorting the data according to labels
df.sort_values("label", inplace=True)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,10295,10296,10297,10298,10299,10300,10301,10302,10303,label
0,43,50,41,58,78,83,67,48,44,46,...,164,164,162,159,156,155,158,153,169,1
8,63,53,35,36,33,34,31,35,39,43,...,169,166,161,158,169,137,41,10,24,1
7,41,47,47,46,44,49,48,58,61,49,...,34,35,34,36,34,39,35,37,38,1
6,44,43,32,32,30,30,38,40,48,66,...,42,42,40,33,29,37,43,43,37,1
5,48,49,45,47,49,57,39,42,53,49,...,44,40,41,49,42,44,47,46,46,1


In [7]:
# seperating images from labels
Y = np.array(df['label'])
df = df.drop(axis=1, labels="label")

In [8]:
df = df.reset_index()
df = df.drop(axis=1, labels="index")
df.head()

,0,1,2,3,4,5,6,7,8,9,...,10294,10295,10296,10297,10298,10299,10300,10301,10302,10303
0,43,50,41,58,78,83,67,48,44,46,...,167,164,164,162,159,156,155,158,153,169
1,63,53,35,36,33,34,31,35,39,43,...,173,169,166,161,158,169,137,41,10,24
2,41,47,47,46,44,49,48,58,61,49,...,27,34,35,34,36,34,39,35,37,38
3,44,43,32,32,30,30,38,40,48,66,...,37,42,42,40,33,29,37,43,43,37
4,48,49,45,47,49,57,39,42,53,49,...,39,44,40,41,49,42,44,47,46,46


In [9]:
# creating the data matrix D
D = [[] for i in range(40)]
for i in range(0, 400): D[i//10].append(df.iloc[i])
D = np.array(D)
print(D.shape)

(40, 10, 10304)


In [10]:
X_train = []
X_test =  []  
labels_train = []
labels_test =  []  

for i in range(0,400):
    if((i%2==0)):
       X_test.append(arr[i,:])
       labels_test.append(labels[i])
    else:
       X_train.append(arr[i,:])
       labels_train.append(labels[i])
 
print(X_test[0])
print(labels_test[0])

[ 43  50  41 ... 158 153 169]
1


In [27]:
def LDA(D, Y=None, noOfDims=None):
    # initializing the variables
    summ, lenn = 0, 0
    # computing the mean for each class
    mu = []
    for x in D: 
        mu.append(np.mean(x, axis=0, keepdims=False).T)
        # calculating the sum and overall number
        # elements for overall mu
        summ += sum(x)
        lenn += len(x)
    mu = np.array(mu)
    overallMu = summ / lenn
    print(mu.shape , overallMu.shape)
    # computing the between class scatter matrices
    SB = mu-overallMu
    SB = np.matmul(SB,SB.T)
    SB = SB * len(D[0])
    print(SB.shape)
    SB = np.sum(SB)
    print(SB.shape)
    # centering the class matrices
    Z = np.array([D[i] - mu[i].T for i in range(len(D))])
    Z = np.reshape(Z, (400,10304))
    # computing the within class scatter matrices
    print(Z.shape)
    S = np.matmul(Z.T, Z)
    print(S.shape)
    # performing eigenvalue decomposition
    S = np.sum(S, axis = 0)
    ###eigValues, eigVectors = np.linalg.eigh(np.matmul(np.linalg.inv(sum(S)), SB))
    # returning the dominant eigen vectors
    ###return eigVectors[::-1][:noOfDims].T

In [12]:
def projectData(X_train, X_test, W):
    X_train = np.array([np.matmul(W.T, X_train[i]) for i in range(X_train.shape[0])])
    X_test = np.array([np.matmul(W.T, X_test[i]) for i in range(X_test.shape[0])])
    return X_train, X_test

In [28]:
W = LDA(D, noOfDims=39)

(40, 10304) (10304,)
(40, 40)
()
(400, 10304)
(10304, 10304)


In [ ]:
# saving the LDA output for later use
file_name = 'LDA_output.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(W, file)
    print(f'Object successfully saved to "{file_name}"')

In [ ]:
projectData(X_train, X_test, W)

In [ ]:
def KMC(X, k, epsilon=0.01, maxIter=100):
  # initializing the error to be max at first
  err = float('inf')
  # randomly initializing the centroids
  centroid = [X[np.random.randint(0, len(X)-1)] for i in range(k)]
  while maxIter > 0 and err >= epsilon:
    maxIter -= 1
    cluster = [[] for i in range(k)]
    # copying the old centroids
    oldCentroid = copy.deepcopy(centroid)
    # assigning each data point to closest cluster
    for x in X:
      minn, j = float('inf'), 0
      for i in range(len(centroid)):
        dist = np.linalg.norm(x-centroid[i]) ** 2
        if dist < minn: minn, j = dist, i
      cluster[j].append(x)
    # updating the centroids
    for i in range(len(cluster)):
      if len(cluster[i]) != 0: centroid[i] = sum(cluster[i]) / len(cluster[i])
    # if centroids didn't change, then terminate
    err = 0
    for i in range(len(centroid)):
      err += np.linalg.norm(centroid[i] - oldCentroid[i]) ** 2
  return cluster